<a href="https://colab.research.google.com/github/FraSab98/Prova/blob/main/Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import csv
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [4]:
from google.colab import files
files.upload()  # Questo ti permette di caricare il file kaggle.json


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"francescosabia","key":"0f56acfa87a8a58ce65a08921381cc20"}'}

In [5]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Imposta i permessi corretti


In [6]:
!kaggle datasets download -d saurabhshahane/fake-news-classification  # Sostituisci <dataset-path> con il path del dataset su Kaggle


Dataset URL: https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification
License(s): Attribution 4.0 International (CC BY 4.0)
fake-news-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!unzip fake-news-classification.zip  # Sostituisci <dataset-file> con il nome del file scaricato

Archive:  fake-news-classification.zip
replace WELFake_Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: WELFake_Dataset.csv     


In [8]:
df = pd.read_csv("WELFake_Dataset.csv")
df.head()


,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72134 non-null  int64 
 1   title       71576 non-null  object
 2   text        72095 non-null  object
 3   label       72134 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB


In [10]:
df['label'].value_counts()

label
1    37106
0    35028
Name: count, dtype: int64

In [11]:
df.shape

(72134, 4)

In [12]:
df.isna().sum()

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [13]:
df = df.dropna() #Handled Missing values by droping those rows

In [14]:
df.isna().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [15]:
df.shape

(71537, 4)

In [16]:
df.reset_index(inplace=True)
df.head()

,index,Unnamed: 0,title,text,label
0,0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
2,3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
3,4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
4,5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


In [17]:
df['title'][0]

'LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]'

In [18]:
df = df.drop(['Unnamed: 0'],axis = 1)
df.head()

,index,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
2,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
3,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
4,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


In [19]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def tokenize_text(text):
    return word_tokenize(text)

# Applica la tokenizzazione
df['tokenized_title'] = df['title'].apply(tokenize_text)
df['tokenized_text'] = df['text'].apply(tokenize_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
df['tokenized_title'][0]

['LAW',
 'ENFORCEMENT',
 'ON',
 'HIGH',
 'ALERT',
 'Following',
 'Threats',
 'Against',
 'Cops',
 'And',
 'Whites',
 'On',
 '9-11By',
 '#',
 'BlackLivesMatter',
 'And',
 '#',
 'FYF911',
 'Terrorists',
 '[',
 'VIDEO',
 ']']

In [21]:
def text_lower(text):
    # Converti il testo in minuscolo
    text = [[word.lower() for word in text.split()] for text in text]
    return text

def remove_special_characters(tokens):
    cleaned_tokens = [re.sub(r'[^a-zA-Z]+',  # Search for all non-letters
                          "",          # Replace all non-letters with spaces
                          str(token)) for token in tokens]
    # Rimuove token vuoti che possono risultare dalla rimozione dei caratteri speciali
    return [token for token in cleaned_tokens if token]

# Scaricare le stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# Funzione per rimuovere le stop words da una lista di token
def remove_stopwords(tokens):
    return [token for token in tokens if token.lower() not in stop_words]

# Funzione combinata per pulire una lista di token
def clean_tokens(tokens):
    tokens = text_lower(tokens)
    tokens_no_special_chars = remove_special_characters(tokens)
    cleaned_tokens = remove_stopwords(tokens_no_special_chars)
    return cleaned_tokens

# Applica la pulizia del testo
df['clean_title'] = df['tokenized_title'].apply(clean_tokens)
df['clean_text'] = df['tokenized_text'].apply(clean_tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
df['clean_title'][0]

['  law  ',
 '  enforcement  ',
 '  on  ',
 '  high  ',
 '  alert  ',
 '  following  ',
 '  threats  ',
 '  against  ',
 '  cops  ',
 '  and  ',
 '  whites  ',
 '  on  ',
 '  9 11by  ',
 '     ',
 '  blacklivesmatter  ',
 '  and  ',
 '     ',
 '  fyf911  ',
 '  terrorists  ',
 '     ',
 '  video  ',
 '     ']

In [26]:
def remove_nan(tokens):
  return [token for token in tokens if token.strip()]

df['clean_title'] = df['clean_title'].apply(remove_nan)
df['clean_text'] = df['clean_text'].apply(remove_nan)

In [27]:
df['clean_title'][0]

['  law  ',
 '  enforcement  ',
 '  on  ',
 '  high  ',
 '  alert  ',
 '  following  ',
 '  threats  ',
 '  against  ',
 '  cops  ',
 '  and  ',
 '  whites  ',
 '  on  ',
 '  9 11by  ',
 '  blacklivesmatter  ',
 '  and  ',
 '  fyf911  ',
 '  terrorists  ',
 '  video  ']

In [28]:
# Inizializzare il PorterStemmer
stemmer = PorterStemmer()

# Funzione per applicare lo stemming a una lista di token
def apply_stemming(tokens):
    return [stemmer.stem(token) for token in tokens]

df['clean_title'] = df['clean_title'].apply(apply_stemming)
df['clean_text'] = df['clean_text'].apply(apply_stemming)

In [29]:
df['clean_title'][0]

['  law  ',
 '  enforcement  ',
 '  on  ',
 '  high  ',
 '  alert  ',
 '  following  ',
 '  threats  ',
 '  against  ',
 '  cops  ',
 '  and  ',
 '  whites  ',
 '  on  ',
 '  9 11by  ',
 '  blacklivesmatter  ',
 '  and  ',
 '  fyf911  ',
 '  terrorists  ',
 '  video  ']